# 1. Dataset

Follow up all the steps for preparing or just use this prepared 'RDD' dataset and directly go to the 'train' chapter. (google drive link)

(1) without image patches, only original country's images.    
RDD2022_all_countries

(2) if you want to use the image patches together, download and prepare these files.     
Japan_640.tar.gz, Norway_640.tar.gz, Norway_1024.tar.gz

And in the google drive link, 'RDD' directory has (1) and (2) both.

(3) You need to prepare the 'RDD_test' directory for submition the final prediction file.    
It can be created by following the script, but also be downloaded by this link.    (RDD_test_link)

# Dataset downloading
Download the dataset for training and testing the object detector.

In [1]:
#%cd GRDDC2022/RoadDamageDetector2022

!pwd

/home/june/RoadDamageDetector


In [ ]:
!wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/RDD2022.zip
!unzip RDD2022.zip

In [ ]:
!cd RDD2022_all_countries

!unzip China_Drone.zip
!unzip China_MotorBike.zip
!unzip Czech.zip
!unzip India.zip
!unzip Japan.zip
!unzip Norway.zip
!unzip United_States.zip

In [ ]:
!cd ..

In [2]:
# !wget RDD2022_all_countries.tar.gz
# !tar xzvf RDD2022_all_countries

# !wget RDD.tar.gz
# !tar xzvf RDD

# !wget RDD_test.tar.gz
# !tar xzvf RDD_test

# Dataset pre-processing
Remove the useless labels in the dataset.

In [6]:
# In the GRDDC2022/RoadDamageDetector2022

%cd RDD2022_all_countries

from sklearn.model_selection import train_test_split

import os
import xml.etree.ElementTree as ET
from shutil import copyfile
import argparse
from pathlib import Path
import random

#!tree -d

[Errno 2] No such file or directory: 'RDD2022_all_countries'
/home/june/RoadDamageDetector/train


- New directory will be created.    
new_countryName
                /Annoatations
                /JPEGImages 

In [31]:
os.getcwd()

# for original countries
countries = ["China_Drone", "China_MotorBike", "Czech", "India", "Japan", "Norway", "United_States"]

# for image patches
#countries = ["Norway_640", "Japan_640", "Norway_1024"]


labels = ["D00", "D10", "D20", "D40"]

for country in countries:
    annoFiles = os.listdir(os.path.join(os.getcwd(), country+"/train/annotations/xmls/"))
    jpgFiles = os.listdir(os.path.join(os.getcwd(), country+"/train/images/"))
    
    newCountry = "new_"+country
    Path(newCountry+"/Annotations").mkdir(parents=True, exist_ok=True)
    Path(newCountry+"/JPEGImages").mkdir(parents=True, exist_ok=True)
    
    for annoFile in annoFiles:
        tree = ET.parse(os.path.join(os.getcwd()+"/"+country+"/train/annotations/xmls/", annoFile))
        root = tree.getroot()
        for obj in root.findall("object"):
            a = obj.find("name").text
            if a not in labels:
                root.remove(obj)
            
        if len(root.findall("object")) > 0:
            tree.write(newCountry+"/Annotations/"+annoFile)
            copyfile(os.path.join(country+"/train/images/", annoFile.split(".")[0])+".jpg", newCountry+"/JPEGImages/"+annoFile.split(".")[0]+".jpg")
        else:
            print(f'{annoFile} no class file')
            continue

- cp all new_countryName/Annotations & JPEGImages into one directory: new_train/Annotations & JPEGImages

In [32]:
%pwd

'/home/june/RoadDamageDetector'

In [35]:
# Copy them into the one directory for chaning to yolo format

Path("new_train/Annotations").mkdir(parents=True, exist_ok=True)
Path("new_train/JPEGImages").mkdir(parents=True, exist_ok=True)

!cp new_China_Drone/Annotations/*.xml new_train/Annotations/
!cp new_China_MotorBike/Annotations/*.xml new_train/Annotations/
!cp new_Czech/Annotations/*.xml new_train/Annotations/
!cp new_India/Annotations/*.xml new_train/Annotations/
!cp new_Japan/Annotations/*.xml new_train/Annotations/
# !cp new_Japan_640/Annotations/*.xml new_train/Annotations/
!cp new_Norway/Annotations/*.xml new_train/Annotations/
# !cp new_Norway_640/Annotations/*.xml new_train/Annotations/
# !cp new_Norway_1024/Annotations/*.xml new_train/Annotations/
!cp new_United_States/Annotations/*.xml new_train/Annotations/

!cp new_China_Drone/JPEGImages/*.jpg new_train/JPEGImages/
!cp new_China_MotorBike/JPEGImages/*.jpg new_train/JPEGImages/
!cp new_Czech/JPEGImages/*.jpg new_train/JPEGImages/
!cp new_India/JPEGImages/*.jpg new_train/JPEGImages/
!cp new_Japan/JPEGImages/*.jpg new_train/JPEGImages/
# !cp new_Japan_640/JPEGImages/*.jpg new_train/JPEGImages/
!cp new_Norway/JPEGImages/*.jpg new_train/JPEGImages/
# !cp new_Norway_640/JPEGImages/*.jpg new_train/JPEGImages/
# !cp new_Norway_1024/JPEGImages/*.jpg new_train/JPEGImages/
!cp new_United_States/JPEGImages/*.jpg new_train/JPEGImages/

In [14]:
# for image patches
# !cp new_Japan_640/Annotations/*.xml new_train/Annotations/
# !cp new_Norway_640/Annotations/*.xml new_train/Annotations/
# !cp new_Norway_1024/Annotations/*.xml new_train/Annotations/

# !cp new_Japan_640/JPEGImages/*.jpg new_train/JPEGImages/
# !cp new_Norway_640/JPEGImages/*.jpg new_train/JPEGImages/
# !cp new_Norway_1024/JPEGImages/*.jpg new_train/JPEGImages/

- Copy all country/test/ into one directory: new_train/test/

In [20]:
Path("RDD_test").mkdir(parents=True, exist_ok=True)

#!cp China_Drone/test/images/*.jpg RDD_test/
!cp China_MotorBike/test/images/*.jpg RDD_test/
!cp Czech/test/images/*.jpg RDD_test/
!cp India/test/images/*.jpg RDD_test/
!cp Japan/test/images/*.jpg RDD_test/
!cp Norway/test/images/*.jpg RDD_test/
!cp United_States/test/images/*.jpg RDD_test/

# Transform to YOLO format from VOC xml

In [25]:
%pwd

'/home/june/RoadDamageDetector'

In [36]:
annoFiles = os.listdir(os.path.join(os.getcwd(), "new_train/Annotations/"))
yoloFile = open("./xml2yolo_damage.txt", "w")
for i in range(len(annoFiles)):
    yoloFile.writelines(os.getcwd() + "/new_train/Annotations/" + annoFiles[i] + "\n")

yoloFile.close()

In [ ]:
!python xml2yolo.py --input-file xml2yolo_damage.txt

- Move all jpg & txt files -> into the one directory: 'RDD' (Road Damage Dataset)

In [38]:
Path("RDD").mkdir(parents=True, exist_ok=True)

# !cp new_train/JPEGImages/*.jpg RDD/
# !cp new_train/labels/*.txt RDD/

/bin/bash: /bin/cp: Argument list too long
/bin/bash: /bin/cp: Argument list too long


- 'cp' issue with too many files, thus use 'find'.

In [41]:
Path("RDD").mkdir(parents=True, exist_ok=True)

!find ./new_train/JPEGImages/ -name "*.jpg" -exec cp {} ./RDD/ \;
!find ./new_train/labels/ -name "*.txt" -exec cp {} ./RDD/ \;

# Split the dataset for training the dataset
Split the dataset to train dataset and validation dataset.

In [19]:
!pwd

/home/june/RoadDamageDetector


In [7]:
# In the train directory

!tree -d

.
└── new_train
    ├── Annotations
    ├── JPEGImages
    └── labels

4 directories


- Without image patches, only original country images.

In [26]:
# single country dataset, example.
new_United_States_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_train/labels/"))
print(len(new_United_States_annoFiles))

United_States_train, United_States_valid = train_test_split(new_United_States_annoFiles, test_size=0.2, random_state=42)
print(len(United_States_train), len(United_States_valid))

file1 = open("US_train.txt", "w")
for i in range(len(United_States_train)):
    file1.writelines('../RoadDamageDetector2022/RDD/' + United_States_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("US_valid.txt", "w")
for i in range(len(United_States_valid)):
    file2.writelines('../RoadDamageDetector2022/RDD/' + United_States_valid[i].split(".")[0] + '.jpg' + "\n")
    
file2.close()

4805
3844 961


- With image patches

In [42]:
# 15 % split for valid5

new_China_1_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_China_Drone/JPEGImages/"))
print(len(new_China_1_annoFiles))

new_China_2_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_China_MotorBike/JPEGImages/"))
print(len(new_China_2_annoFiles))

new_Czech_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_Czech/JPEGImages/"))
print(len(new_Czech_annoFiles))

new_India_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_India/JPEGImages/"))
print(len(new_India_annoFiles))

new_Japan_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_Japan/JPEGImages/"))
print(len(new_Japan_annoFiles))

new_Japan_640_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_Japan_640/JPEGImages/"))
print(len(new_Japan_640_annoFiles))

new_Norway_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_Norway/JPEGImages/"))
print(len(new_Norway_annoFiles))

new_Norway_640_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_Norway_640/JPEGImages/"))
print(len(new_Norway_640_annoFiles))

new_Norway_1024_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_Norway_1024/JPEGImages/"))
print(len(new_Norway_1024_annoFiles))

new_US_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_United_States/JPEGImages/"))
print(len(new_US_annoFiles))

lenOfChina1 = len(new_China_1_annoFiles)
lenOfChina2 = len(new_China_2_annoFiles)
lenOfCzech = len(new_Czech_annoFiles)
lenOfIndia = len(new_India_annoFiles)
lenOfJapan = len(new_Japan_annoFiles)
lenOfJapan640 = len(new_Japan_640_annoFiles)
lenOfNorway = len(new_Norway_annoFiles)
lenOfNorway640 = len(new_Norway_640_annoFiles)
lenOfNorway1024 = len(new_Norway_1024_annoFiles)
lenOfUS = len(new_US_annoFiles)


China_1_train, China_1_valid = new_China_1_annoFiles[:int(-lenOfChina1*0.15)], new_China_1_annoFiles[int(-lenOfChina1*0.15):]

China_2_train, China_2_valid = new_China_2_annoFiles[:int(-lenOfChina2*0.15)], new_China_2_annoFiles[int(-lenOfChina2*0.15):]

Czech_train, Czech_valid = new_Czech_annoFiles[:int(-lenOfCzech*0.15)], new_Czech_annoFiles[int(-lenOfCzech*0.15):]

India_train, India_valid = new_India_annoFiles[:int(-lenOfIndia*0.15)], new_India_annoFiles[int(-lenOfIndia*0.15):]

Japan_train, Japan_valid = new_Japan_annoFiles[:int(-lenOfJapan*0.15)], new_Japan_annoFiles[int(-lenOfJapan*0.15):]

Japan_640_train, Japan_640_valid = new_Japan_640_annoFiles[:int(-lenOfJapan640*0.15)], new_Japan_640_annoFiles[int(-lenOfJapan640*0.15):]

Norway_train, Norway_valid = new_Norway_annoFiles[:int(-lenOfNorway640*0.15)], new_Norway_annoFiles[int(-lenOfNorway*0.15):]

Norway_640_train, Norway_640_valid = new_Norway_640_annoFiles[:int(-lenOfNorway640*0.15)], new_Norway_640_annoFiles[int(-lenOfNorway640*0.15):]

Norway_1024_train, Norway_1024_valid = new_Norway_1024_annoFiles[:int(-lenOfNorway1024*0.15)], new_Norway_1024_annoFiles[int(-lenOfNorway1024*0.15):]

US_train, US_valid = new_US_annoFiles[:int(-lenOfUS*0.15)], new_US_annoFiles[int(-lenOfUS*0.15):]

1919
1934
1072
3223
7900
515
2914
16933
31218
4805


In [20]:
print(len(China_1_train), len(China_1_valid))


print(len(China_2_train), len(China_2_valid))


print(len(Czech_train), len(Czech_valid))


print(len(India_train), len(India_valid))


print(len(Japan_train), len(Japan_valid))


print(len(Japan_train), len(Japan_valid))


print(len(Norway_train), len(Norway_valid))


print(len(Norway_640_train), len(Norway_640_valid))

print(len(Norway_1024_train), len(Norway_1024_valid))

print(len(US_train), len(US_valid))

1632 287
1644 290
912 160
2740 483
6715 1185
6715 1185
375 437
14394 2539
4085 720


In [43]:
# mix source training dataset for valid5

#all_train = new_China_1_annoFiles + new_China_2_annoFiles + new_Czech_annoFiles + new_India_annoFiles + new_Japan_annoFiles + new_Japan_640_annoFiles + new_Norway_annoFiles + new_Norway_640_annoFiles + new_US_annoFiles
#random.shuffle(CzInJa_train)
all_train = China_1_train + China_2_train + Czech_train + India_train + Japan_train + Japan_640_train + Norway_train + Norway_640_train +  Norway_1024_train + US_train

all_valid = China_1_valid + China_2_valid + Czech_valid + India_valid + Japan_valid + Japan_640_valid + Norway_valid + Norway_640_valid + Norway_1024_valid + US_valid



file1 = open("./all_train_640_1024_15_5.txt", "w")
for i in range(len(all_train)):
    file1.writelines('../RoadDamageDetector2022/RDD/' + all_train[i] + "\n")

file1.close()

file2 = open("./all_valid_640_1024_15_5.txt", "w")
for i in range(len(all_valid)):
    file2.writelines('../RoadDamageDetector2022/RDD/' + all_valid[i] + "\n")

file2.close()

# 2. Train
I did various experiments, and could get some good models.    
This is just an example, there were many mixed dataset,    
and failed weight files, too.

We used YOLOv5x P5 and P6 models.
thus you need to change some settings.
(1) data.yaml, (2) hyp.yaml, (3) model.cfg    

and download a pre-trained weight of IMSC (GRDDC 2020 winner). (https://github.com/USC-InfoLab/rddc2020)

In [ ]:
!cd ../yolov5

In [ ]:
# for YOLOv5x P5 model
!python train.py --img 640 --batch-size 32 --epochs 200 --data data/road.yaml --cfg models/yolov5x_road.yaml --weights last_95_640_16.pt --hyp data/hyps/hyp.scratch.road.yaml --name x_ --workers 8 --cache

# for YOLOv5x P6 model
!python train.py --img 640 --batch-size 16 --epochs 200 --data data/road.yaml --cfg models/hub/yolov5x6_road.yaml --weights last_95_640_16.pt --hyp data/hyps/hyp.scratch.road.yaml --name x_ --workers 8 --cache

- if CUDA error has cuased, change the pytorch version.
- then execute it again.

In [ ]:
conda install pytorch==1.9.1 torchvision==0.10.1 torchaudio==0.9.1 cudatoolkit=11.3 -c pytorch -c conda-forge

# 3. Inference
Maybe you need the trained weight files.    
We trained and used 12 YOLOv5 P5 and P6 models, and you can get it from here.    
(https://drive.google.com/file/d/1tGlmF7Om5aUfBh-Jsu1jw_JQ-AcAuv7P/view?usp=sharing)    

In [10]:
# China_MotorBike
python detect.py --weights weights/x6_15_5_640_3.pt  weights/x_640_.pt weights/x_nw_1024_6.pt weights/x_16.pt weights/x_us_3.pt weights/x_nw_1024_5.pt weights/x_15_5_640_.pt weights/x_15_5_640_4.pt weights/x_15_5_640_1024_.pt weights/x6_15_1_640_1024_2.pt weights/x6_15_5_640_1024_3.pt weights/x6_15_5_640_1024_4.pt --source ../RoadDamageDetector2022/RDD2022_all_countries/China_MotorBike/test/images/ --imgsz 640 --conf-thres 0.3 --iou-thres 0.999 --save-txt --save-conf --augment --agnostic-nms --nosave
# Czech
python detect.py --weights weights/x6_15_5_640_3.pt  weights/x_640_.pt weights/x_nw_1024_6.pt weights/x_16.pt weights/x_us_3.pt weights/x_nw_1024_5.pt weights/x_15_5_640_.pt weights/x_15_5_640_4.pt weights/x_15_5_640_1024_.pt weights/x6_15_1_640_1024_2.pt weights/x6_15_5_640_1024_3.pt weights/x6_15_5_640_1024_4.pt --source ../RoadDamageDetector2022/RDD2022_all_countries/Czech/test/images/ --imgsz 640 --conf-thres 0.3 --iou-thres 0.999 --save-txt --save-conf --augment --agnostic-nms --nosave
# India
python detect.py --weights weights/x6_15_5_640_3.pt  weights/x_640_.pt weights/x_nw_1024_6.pt weights/x_16.pt weights/x_us_3.pt weights/x_nw_1024_5.pt weights/x_15_5_640_.pt weights/x_15_5_640_4.pt weights/x_15_5_640_1024_.pt weights/x6_15_1_640_1024_2.pt weights/x6_15_5_640_1024_3.pt weights/x6_15_5_640_1024_4.pt --source ../RoadDamageDetector2022/RDD2022_all_countries/India/test/images/ --imgsz 768 --conf-thres 0.4 --iou-thres 0.999 --save-txt --save-conf --augment --agnostic-nms --nosave
# Japan
python detect.py --weights weights/x6_15_5_640_3.pt  weights/x_640_.pt weights/x_nw_1024_6.pt weights/x_16.pt weights/x_us_3.pt weights/x_nw_1024_5.pt weights/x_15_5_640_.pt weights/x_15_5_640_4.pt weights/x_15_5_640_1024_.pt weights/x6_15_1_640_1024_2.pt weights/x6_15_5_640_1024_3.pt weights/x6_15_5_640_1024_4.pt --source ../RoadDamageDetector2022/RDD2022_all_countries/Japan/test/images/ --imgsz 640 --conf-thres 0.4 --iou-thres 0.999 --save-txt --save-conf --augment --agnostic-nms --nosave
# Norway
python detect.py --weights weights/x6_15_5_640_3.pt  weights/x_640_.pt weights/x_nw_1024_6.pt weights/x_16.pt weights/x_us_3.pt weights/x_nw_1024_5.pt weights/x_15_5_640_.pt weights/x_15_5_640_4.pt weights/x_15_5_640_1024_.pt weights/x6_15_1_640_1024_2.pt weights/x6_15_5_640_1024_3.pt weights/x6_15_5_640_1024_4.pt --source ../RoadDamageDetector2022/RDD2022_all_countries/Norway/test/images/ --imgsz 1280 --conf-thres 0.4 --iou-thres 0.999 --save-txt --save-conf --augment --agnostic-nms --nosave
# United_States
python detect.py --weights weights/x6_15_5_640_3.pt  weights/x_640_.pt weights/x_nw_1024_6.pt weights/x_16.pt weights/x_us_3.pt weights/x_nw_1024_5.pt weights/x_15_5_640_.pt weights/x_15_5_640_4.pt weights/x_15_5_640_1024_.pt weights/x6_15_1_640_1024_2.pt weights/x6_15_5_640_1024_3.pt weights/x6_15_5_640_1024_4.pt --source ../RoadDamageDetector2022/RDD2022_all_countries/United_States/test/images/ --imgsz 640 --conf-thres 0.4 --iou-thres 0.999 --save-txt --save-conf --augment --agnostic-nms --nosave

# or use bash file
# chmod u+x submission_batch.sh
# ./submission_batch.sh

# 4. Merge
Merging the final outputs is in the yolov5/merge.ipynb